# Pickling cuML Models for Persistence

This notebook demonstrates simple pickling of both single-GPU and multi-GPU cuML models for persistence

In [1]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

## Single GPU Model Pickling

All single-GPU estimators are pickleable. The following example demonstrates the creation of a synthetic dataset, training, and pickling of the resulting model for storage. Trained single-GPU models can also be used to distribute the inference on a Dask cluster, which the `Distributed Model Pickling` section below demonstrates.

In [2]:
from cuml.datasets import make_blobs

X, y = make_blobs(n_samples=50,
                  n_features=10,
                  centers=5,
                  cluster_std=0.4,
                  random_state=0)

In [3]:
from cuml.cluster import KMeans

model = KMeans(n_clusters=5)

model.fit(X)

KMeans()

In [4]:
import pickle

pickle.dump(model, open("kmeans_model.pkl", "wb"))

In [5]:
model = pickle.load(open("kmeans_model.pkl", "rb"))

In [6]:
model.cluster_centers_

array([[ 4.6749854 ,  8.213466  , -9.075721  ,  9.568374  ,  8.454808  ,
        -1.2327975 ,  3.390371  , -7.8282413 , -0.8454461 ,  0.62885725],
       [-4.243999  ,  5.610707  , -5.6697764 , -1.7957246 , -9.255528  ,
         0.7177438 ,  4.4435897 , -2.8747153 , -5.0900965 ,  9.684121  ],
       [ 5.2615476 , -4.0487256 ,  4.464928  , -2.9367518 ,  3.5061095 ,
        -4.016832  , -3.463885  ,  6.078449  , -6.953326  , -1.004144  ],
       [-3.008261  ,  4.6259604 , -4.483249  ,  2.228457  ,  1.643532  ,
        -2.4505193 , -5.258201  , -1.6679403 , -7.985754  ,  2.8311467 ],
       [-5.6072407 ,  2.2695985 , -3.7516537 , -1.8182003 , -5.143028  ,
         7.599363  ,  2.8252366 ,  8.773042  ,  1.6198314 ,  1.1772048 ]],
      dtype=float32)

## Distributed Model Pickling

The distributed estimator wrappers inside of the `cuml.dask` are not intended to be pickled directly. The Dask cuML estimators provide a function `get_combined_model()`, which returns the trained single-GPU model for pickling. The combined model can be used for inference on a single-GPU, and the `ParallelPostFit` wrapper from the [Dask-ML](https://ml.dask.org/meta-estimators.html) library can be used to perform distributed inference on a Dask cluster.

In [7]:
from dask.distributed import Client
from dask_cuda import LocalCUDACluster

cluster = LocalCUDACluster()
client = Client(cluster)
client

2022-05-28 10:44:05,705 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize


Connection method: Cluster object,Cluster type: dask_cuda.LocalCUDACluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 1
Total threads: 1,Total memory: 251.79 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:46643,Workers: 1
Dashboard: http://127.0.0.1:8787/status,Total threads: 1
Started: Just now,Total memory: 251.79 GiB
Comm: tcp://127.0.0.1:41827,Total threads: 1
Dashboard: http://127.0.0.1:36785/status,Memory: 251.79 GiB
Nanny: tcp://127.0.0.1:38557,


In [8]:
from cuml.dask.datasets import make_blobs

n_workers = len(client.scheduler_info()["workers"].keys())

X, y = make_blobs(n_samples=5000, 
                  n_features=30,
                  centers=5, 
                  cluster_std=0.4, 
                  random_state=0,
                  n_parts=n_workers*5)

X = X.persist()
y = y.persist()

In [9]:
from cuml.dask.cluster import KMeans

dist_model = KMeans(n_clusters=5)

In [10]:
dist_model.fit(X)

2022-05-28 10:44:12,040 - distributed.worker - WARNING - Compute Failed
Key:       _get_model_attr-9985085f-514b-4053-b78b-3d8153887c96
Function:  _get_model_attr
args:      (KMeansMG(), '_ipython_canary_method_should_not_exist_')
kwargs:    {}
Exception: 'AttributeError("Attribute _ipython_canary_method_should_not_exist_ does not exist on model <class \'cuml.cluster.kmeans_mg.KMeansMG\'>")'

2022-05-28 10:44:12,062 - distributed.worker - WARNING - Compute Failed
Key:       _get_model_attr-45e08452-c5a4-4968-9967-862c91830ba7
Function:  _get_model_attr
args:      (KMeansMG(), '_ipython_display_')
kwargs:    {}
Exception: 'AttributeError("Attribute _ipython_display_ does not exist on model <class \'cuml.cluster.kmeans_mg.KMeansMG\'>")'

2022-05-28 10:44:12,083 - distributed.worker - WARNING - Compute Failed
Key:       _get_model_attr-125bb8f7-83c4-4815-9bd2-a153f96d0323
Function:  _get_model_attr
args:      (KMeansMG(), '_ipython_canary_method_should_not_exist_')
kwargs:    {}
Exception

2022-05-28 10:44:12,244 - distributed.worker - WARNING - Compute Failed
Key:       _get_model_attr-a30b3207-8d06-437d-a778-e2f2aeb93626
Function:  _get_model_attr
args:      (KMeansMG(), '_repr_pdf_')
kwargs:    {}
Exception: 'AttributeError("Attribute _repr_pdf_ does not exist on model <class \'cuml.cluster.kmeans_mg.KMeansMG\'>")'

2022-05-28 10:44:12,258 - distributed.worker - WARNING - Compute Failed
Key:       _get_model_attr-f72554db-9603-4af9-ac65-7d907c7752c3
Function:  _get_model_attr
args:      (KMeansMG(), '_ipython_canary_method_should_not_exist_')
kwargs:    {}
Exception: 'AttributeError("Attribute _ipython_canary_method_should_not_exist_ does not exist on model <class \'cuml.cluster.kmeans_mg.KMeansMG\'>")'

2022-05-28 10:44:12,279 - distributed.worker - WARNING - Compute Failed
Key:       _get_model_attr-86540153-ba7a-4ee3-823e-ae2798b43542
Function:  _get_model_attr
args:      (KMeansMG(), '_repr_jpeg_')
kwargs:    {}
Exception: 'AttributeError("Attribute _repr_jpeg_ do

In [11]:
import pickle

single_gpu_model = dist_model.get_combined_model()
pickle.dump(single_gpu_model, open("kmeans_model.pkl", "wb"))

In [12]:
single_gpu_model = pickle.load(open("kmeans_model.pkl", "rb"))

In [13]:
single_gpu_model.cluster_centers_

array([[-4.6475401e+00, -9.5672169e+00,  6.6923518e+00,  4.4359231e+00,
         2.1902735e+00,  2.5834754e+00,  5.9448940e-01,  6.2568665e+00,
        -8.7821636e+00, -4.1232011e-01,  9.8151760e+00,  7.5641565e+00,
         1.0003010e+01, -5.8680429e+00, -1.2743108e+00, -2.5393455e+00,
        -1.0847501e+00, -5.2629204e+00, -9.3071022e+00,  4.6179361e+00,
        -9.7068951e-02, -3.9351206e+00,  6.1767278e+00, -7.4346890e+00,
         5.6496072e+00, -8.5544834e+00, -7.5265579e+00, -5.5195608e+00,
         4.8197637e+00,  2.5235438e+00],
       [ 6.2794294e+00,  9.2293949e+00,  8.3403702e+00,  9.0330515e+00,
         7.6893487e+00, -9.9538219e-01, -6.2780762e+00,  1.3599334e+00,
        -6.9744482e+00, -5.9463463e+00,  1.0695117e+00, -8.0422480e-03,
         2.8183143e+00,  1.8317668e+00, -8.2557344e+00,  3.0514317e+00,
        -8.4958019e+00,  9.7238483e+00, -7.7455082e+00,  3.4521689e+00,
        -3.9248335e+00, -4.1106420e+00,  2.6693094e+00,  1.2985628e+00,
         1.0421574e+00,

## Exporting cuML Random Forest models for inferencing on machines without GPUs

Starting with cuML version 21.06, you can export cuML Random Forest models and run predictions with them on machines without an NVIDIA GPUs. The [Treelite](https://github.com/dmlc/treelite) package defines an efficient exchange format that lets you portably move the cuML Random Forest models to other machines. We will refer to the exchange format as "checkpoints."

Here are the steps to export the model:

1. Call `to_treelite_checkpoint()` to obtain the checkpoint file from the cuML Random Forest model.

In [14]:
from cuml.ensemble import RandomForestClassifier as cumlRandomForestClassifier
from sklearn.datasets import load_iris
import numpy as np

X, y = load_iris(return_X_y=True)
X, y = X.astype(np.float32), y.astype(np.int32)
clf = cumlRandomForestClassifier(max_depth=3, random_state=0, n_estimators=10)
clf.fit(X, y)

checkpoint_path = './checkpoint.tl'
# Export cuML RF model as Treelite checkpoint
clf.convert_to_treelite_model().to_treelite_checkpoint(checkpoint_path)

/opt/conda/envs/rapids/lib/python3.9/site-packages/cuml/internals/api_decorators.py:794: UserWarning: For reproducible results in Random Forest Classifier or for almost reproducible results in Random Forest Regressor, n_streams=1 is recommended. If n_streams is > 1, results may vary due to stream/thread timing differences, even when random_state is set
  return func(**kwargs)


2. Copy the generated checkpoint file `checkpoint.tl` to another machine on which you'd like to run predictions.

3. On the target machine, install Treelite by running `pip install treelite` or `conda install -c conda-forge treelite`. The machine does not need to have an NVIDIA GPUs and does not need to have cuML installed.

4. You can now load the model from the checkpoint, by running the following on the target machine:

In [15]:
import treelite

# The checkpoint file has been copied over
checkpoint_path = './checkpoint.tl'
tl_model = treelite.Model.deserialize(checkpoint_path)
out_prob = treelite.gtil.predict(tl_model, X, pred_margin=True)
print(out_prob)

[[1.         0.         0.        ]
 [1.         0.         0.        ]
 [1.         0.         0.        ]
 [1.         0.         0.        ]
 [1.         0.         0.        ]
 [1.         0.         0.        ]
 [1.         0.         0.        ]
 [1.         0.         0.        ]
 [1.         0.         0.        ]
 [1.         0.         0.        ]
 [1.         0.         0.        ]
 [1.         0.         0.        ]
 [1.         0.         0.        ]
 [1.         0.         0.        ]
 [1.         0.         0.        ]
 [1.         0.         0.        ]
 [1.         0.         0.        ]
 [1.         0.         0.        ]
 [0.9        0.1        0.        ]
 [1.         0.         0.        ]
 [1.         0.         0.        ]
 [1.         0.         0.        ]
 [1.         0.         0.        ]
 [1.         0.         0.        ]
 [1.         0.         0.        ]
 [1.         0.         0.        ]
 [1.         0.         0.        ]
 [1.         0.         0.  